In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import mediapipe as mp
import cv2
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import os
from tensorflow.keras.models import load_model

In [5]:
# Expressions labels
expressions = ["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
train_data_dir = '/content/Dataset/train/'
test_data_dir = '/content/Dataset/test/'

In [6]:
# Moving window for smoothing predictions
PREDICTION_WINDOW = deque(maxlen=5)

In [7]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False, max_num_faces=5, min_detection_confidence=0.5
)

In [8]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, VGG19


def build_model(type):
    """Build the CNN model."""
    #  Load VGG19 with pretrained ImageNet weights
    if type == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
    else:
        base_model = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


    # Freeze the base layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    output = Dense(7, activation='softmax')(x)  # Example: 10 classes

    # Create new model
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model
# build_model()

In [9]:
def train_model(tp,model, train_dir, test_dir, epochs=50):
    """Train the model with given data."""
    # Data augmentation
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=30,
        shear_range=0.3,
        zoom_range=0.3,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
    )

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        color_mode="rgb",
        target_size=(48, 48),
        batch_size=32,
        class_mode="categorical",
        shuffle = True
    )
    validation_generator = test_datagen.flow_from_directory(
        test_data_dir,
        color_mode="rgb",
        target_size=(48,48),
        batch_size=32,
        class_mode="categorical",
        shuffle = True
    )

    num_train_imgs = 0
    for root, dirs, files in os.walk(train_data_dir):
        num_train_imgs += len(files)
    num_test_imgs = 0
    for root, dirs, files in os.walk(test_data_dir):
        num_test_imgs += len(files)

    print(num_train_imgs)
    print(num_test_imgs)

    history = model.fit(
        train_generator,
        steps_per_epoch=num_train_imgs // 32,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_test_imgs // 32,

    )

    model.save(f"model_{tp}.h5")

    return history

In [10]:
def preprocess_face(face_img):
    """Preprocess a face image for prediction."""
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    normalized = resized / 255.0
    return normalized.reshape(1, 48, 48, 1)

In [11]:
def detect_expression(frame, model):
    """Detect facial expression in a video frame."""
    height, width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            x_min = y_min = float("inf")
            x_max = y_max = float("-inf")

            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * width), int(landmark.y * height)
                x_min = min(x_min, x)
                x_max = max(x_max, x)
                y_min = min(y_min, y)
                y_max = max(y_max, y)

            padding = 20
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(width, x_max + padding)
            y_max = min(height, y_max + padding)

            face = frame[y_min:y_max, x_min:x_max]
            if face.size == 0:
                continue

            processed_face = preprocess_face(face)
            prediction = model.predict(processed_face)[0]
            PREDICTION_WINDOW.append(prediction)

            avg_prediction = np.mean(PREDICTION_WINDOW, axis=0)
            expression = expressions[np.argmax(avg_prediction)]
            confidence = float(avg_prediction.max())

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"{expression} ({confidence:.2f})",
                (x_min, y_min - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.9,
                (0, 255, 0),
                2,
            )

    return frame

In [12]:
def run_realtime(model):
    """Run real-time facial expression detection."""
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        processed_frame = detect_expression(frame, model)

        cv2.imshow("Facial Expression Detection", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [13]:
def evaluate_model(model, test_dir, history):
    """Evaluate the model and visualize the results."""
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode="categorical",
        color_mode="grayscale",
        shuffle=False,
    )

    # Get predictions and calculate metrics
    predictions = model.predict(test_generator)
    true_labels = test_generator.classes
    predicted_labels = np.argmax(predictions, axis=1)

    cm = confusion_matrix(true_labels, predicted_labels)
    report = classification_report(
        true_labels, predicted_labels, target_names=expressions
    )
    accuracy = np.sum(true_labels == predicted_labels) / len(true_labels)

    print(f"Overall Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(report)

    # Plot Confusion Matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=expressions,
        yticklabels=expressions,
    )
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

    # Plot Accuracy and Loss over Epochs
    plt.figure(figsize=(12, 4))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [14]:
def main():

    types = ['vgg16','vgg19']
    for tp in types:
      print(f"Model type: {tp}")
      # Build the model
      model = build_model(tp)

      # Train the model (optional, you can load a pretrained model)
      train = input("Do you want to train the model? (y/n): ")
      if train.lower() == 'y':
          history = train_model(tp,model, train_data_dir, test_data_dir, epochs=60)
      else:
          # Load pre-trained model (if available)
          if os.path.exists(f"model_{tp}.h5"):
              model = load_model("model_CNN.h5")
              print("Model loaded successfully.")
          else:
              print("No pre-trained model found. You should train the model first.")
              return

      # Evaluate the model on the test set
      evaluate = input("Do you want to evaluate the model? (y/n): ")
      if evaluate.lower() == 'y':
          evaluate_model(model, test_data_dir, history)

      # Run real-time facial expression detection
      run_realtime_choice = input("Do you want to run real-time facial expression detection? (y/n): ")
      if run_realtime_choice.lower() == 'y':
          run_realtime(model)

In [ ]:
if __name__ == "__main__":
    main()

Model type: vgg16
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
28709
7178
Epoch 1/60


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 52s 53ms/step - accuracy: 0.2830 - loss: 1.7595 - val_accuracy: 0.3418 - val_loss: 1.6607
Epoch 2/60
  1/897 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.3438 - loss: 1.7145

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3438 - loss: 1.7145 - val_accuracy: 0.3417 - val_loss: 1.6638
Epoch 3/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 82s 60ms/step - accuracy: 0.3214 - loss: 1.6871 - val_accuracy: 0.3612 - val_loss: 1.6234
Epoch 4/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4062 - loss: 1.5622 - val_accuracy: 0.3643 - val_loss: 1.6220
Epoch 5/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 45s 50ms/step - accuracy: 0.3370 - loss: 1.6671 - val_accuracy: 0.3637 - val_loss: 1.6150
Epoch 6/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4062 - loss: 1.7025 - val_accuracy: 0.3608 - val_loss: 1.6200
Epoch 7/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - accuracy: 0.3384 - loss: 1.6629 - val_accuracy: 0.3774 - val_loss: 1.5906
Epoch 8/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4688 - loss: 1.6942 - val_accuracy: 0.3775 - val_loss: 1.5901
Epoch 9/60
897/897 ━━━━━━━━━━━━━━━━━━━━ 43s 48ms/step - accuracy: 0.3432 - loss: 1.6519 - val_accuracy: 0.366